In [2]:
import pandas as pd
import numpy as np
import importlib
from brain_age_prediction import utils, viz
importlib.reload(utils)
importlib.reload(viz)

<module 'brain_age_prediction.viz' from '/home/laurar/brain_age_1D-CNN/src/brain_age_prediction/viz.py'>

Load saved overview of IDs, metadata (and new model predictions) and combine them with Ridge predictions + BAGs:

In [3]:
# save path for predictions
preds_path = '/ritter/share/projects/laura_riedel_thesis/predictions/'
# load data overview
data_overview_full = pd.read_csv(preds_path+'predictions_overview_100-500p.csv')
# limit to IDs present in heldout test set
data_overview_heldout = data_overview_full[data_overview_full['split']=='heldout_test'].copy()
data_overview_heldout.reset_index(inplace=True,drop=True)
# load ridge predictions
ridge_df = pd.read_csv(preds_path+'ridge_preds_100-500p.csv')
# calculate BAGs
ridge_df = utils.calculate_bag(ridge_df, models=['ridge'])
# linearly detrend BAGs
ridge_df = utils.detrend_bag(ridge_df, models=['ridge'])
# merge ridge info with overview
data_overview_heldout = data_overview_heldout.merge(ridge_df, how='outer', on=['eid','age'], copy=False)
data_overview_heldout

,eid,age,split,bmi,digit substitution,education,fluid intelligence,grip,depressive episode,all depression,...,genetic pc 3,batch_nb_orig,predicted_age_orig,batch_nb_new,predicted_age_new,bag_orig,bag_new,predicted_age_ridge,bag_ridge,bag_ridge_detrended
0,1000030,65,heldout_test,21.2569,NaN,20.0,8.0,18.0,1.0,1.0,...,-0.344172,1.0,64.256958,1.0,71.693298,-0.743042,6.693298,65.442215,0.442215,1.083458
1,1000059,68,heldout_test,NaN,22.0,20.0,7.0,NaN,0.0,0.0,...,-1.093500,18.0,58.876446,18.0,58.413288,-9.123554,-9.586712,65.902210,-2.097790,0.257545
2,1000077,58,heldout_test,33.4834,18.0,19.0,3.0,48.0,NaN,NaN,...,0.260200,34.0,63.836048,34.0,66.193604,5.836048,8.193604,60.723910,2.723910,-0.634395
3,1000100,57,heldout_test,23.4552,NaN,20.0,NaN,28.0,0.0,0.0,...,-4.452460,2.0,55.072990,2.0,57.406872,-1.927010,0.406872,51.595610,-5.404390,-9.334059
4,1000202,67,heldout_test,26.6846,19.0,20.0,6.0,45.0,NaN,NaN,...,-4.424320,16.0,64.058601,16.0,67.427048,-2.941399,0.427048,62.639080,-4.360920,-2.576949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2035762,73,heldout_test,25.3688,12.0,7.0,5.0,24.0,0.0,0.0,...,-2.457690,7.0,77.852402,7.0,74.190857,4.852402,1.190857,70.628130,-2.371870,2.840285
4996,2264650,61,heldout_test,30.5389,NaN,7.0,NaN,41.0,0.0,0.0,...,-3.327250,29.0,61.950256,29.0,63.557865,0.950256,2.557865,60.719040,-0.280960,-1.925173
4997,2329354,68,heldout_test,22.5614,18.0,20.0,8.0,30.0,0.0,0.0,...,NaN,36.0,66.802460,36.0,65.315231,-1.197540,-2.684769,66.051830,-1.948170,0.407165
4998,2457356,60,heldout_test,29.6270,18.0,20.0,8.0,22.0,1.0,1.0,...,NaN,9.0,61.610027,9.0,63.635746,1.610027,3.635746,58.524540,-1.475460,-3.691037


In [14]:
true_age_corrs = viz.preds_corr_overview(data_overview_heldout, variables=False, models=['ridge'])
true_age_corrs

,True age vs.,Corr
0,Predicted age ridge model,0.667995
1,BAG ridge model,-0.762226
2,Detrended BAG ridge model,0.000579


In [21]:
variables_corrs = viz.preds_corr_overview(data_overview_heldout, variables=True, models=['ridge'])
variables_corrs

,Variable,Corr BAG ridge model,Corr detrended BAG ridge model
0,bmi,0.041941,0.050598
1,digit substitution,0.309093,-0.040593
2,education,0.051927,-0.045850
3,fluid intelligence,0.031276,-0.046152
4,grip,0.131039,0.045601
5,depressive episode,0.066166,0.013508
6,all depression,0.068684,0.013904
7,recurrent depressive disorder,0.053814,0.010894
8,multiple sclerosis,0.037442,0.023234
9,sex,-0.036003,0.067829


In [22]:
# save overviews as LaTeX table
# table_path = '../../viz/'
# true_age_corrs.to_latex(table_path+'table_true_age_corrs_ridge.tex',
#                          index=False,
#                          float_format='{:.4f}'.format,
#                          column_format='lrrrr',
#                          bold_rows=True)
# variables_corrs.to_latex(table_path+'table_variables_corrs_ridge.tex',
#                          index=False,
#                          float_format='{:.4f}'.format,
#                          column_format='lrrrr',
#                          bold_rows=True)

/tmp/ipykernel_3848546/3274609115.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  true_age_corrs.to_latex(table_path+'table_true_age_corrs_ridge.tex',
/tmp/ipykernel_3848546/3274609115.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  variables_corrs.to_latex(table_path+'table_variables_corrs_ridge.tex',


Using bootstrapping for confidence interval of correlations:

In [4]:
true_age_corrs_bs, variables_corrs_bs = utils.bootstrap_pipeline(data_overview_heldout,models=['ridge'],n_iterations=10)
true_age_corrs_bs

,True age vs.,Corr mean,Corr std
0,Predicted age ridge model,0.662862,0.006874
1,BAG ridge model,-0.760444,0.004981
2,Detrended BAG ridge model,-0.001744,0.008584


In [6]:
true_age_corrs_bs_100, variables_corrs_bs_100 = utils.bootstrap_pipeline(data_overview_heldout,models=['ridge'],n_iterations=100)
true_age_corrs_bs_100

,True age vs.,Corr mean,Corr std
0,Predicted age ridge model,0.667449,0.008035
1,BAG ridge model,-0.762438,0.006342
2,Detrended BAG ridge model,-0.000241,0.013452


In [5]:
variables_corrs_bs

,Variable,Corr BAG ridge model mean,Corr BAG ridge model std,Corr detrended BAG ridge model mean,Corr detrended BAG ridge model std
0,bmi,0.039271,0.012866,0.047203,0.014912
1,digit substitution,0.311713,0.011933,-0.040520,0.016124
2,education,0.055167,0.011668,-0.044493,0.015149
3,fluid intelligence,0.030096,0.010593,-0.045872,0.010622
4,grip,0.139286,0.009524,0.048818,0.014410
5,depressive episode,0.066543,0.009730,0.015620,0.009622
6,all depression,0.068583,0.009334,0.015599,0.009105
7,recurrent depressive disorder,0.056389,0.015901,0.018865,0.018433
8,multiple sclerosis,0.041183,0.012570,0.024518,0.016187
9,sex,-0.029305,0.011906,0.070369,0.012507


In [7]:
variables_corrs_bs_100

,Variable,Corr BAG ridge model mean,Corr BAG ridge model std,Corr detrended BAG ridge model mean,Corr detrended BAG ridge model std
0,bmi,0.041452,0.014054,0.051921,0.014008
1,digit substitution,0.313747,0.016473,-0.036129,0.016491
2,education,0.053113,0.012788,-0.045844,0.013213
3,fluid intelligence,0.031331,0.014858,-0.044679,0.015591
4,grip,0.130267,0.014740,0.044696,0.015349
5,depressive episode,0.065941,0.015863,0.013193,0.015121
6,all depression,0.068111,0.015851,0.013296,0.015039
7,recurrent depressive disorder,0.052404,0.018447,0.008946,0.017802
8,multiple sclerosis,0.035616,0.017459,0.021731,0.015681
9,sex,-0.037019,0.015314,0.065455,0.014771
